### Gene Correlation Matrix: Analyzing Expression Data in AnnData Format

This Jupyter Notebook demonstrates how to download and analyze gene expression data in AnnData format based on a specified tissue or cell type. It filters out genes with zero expression values and computes a correlation matrix for a specified gene.

#### Steps:

1. **Open and Query Data:**
   - Access gene expression data using `cellxgene_census.open_soma()`.
   - Query data for 'Homo sapiens' and filter by cell type ('adipocyte') or tissue type ('Adipose').

2. **Filter Genes:**
   - Set gene names (`adata.var['feature_id']`).
   - Filter out genes with zero expression values.

3. **Convert to DataFrame:**
   - Convert the filtered expression data into a Pandas DataFrame.

4. **Filter Samples:**
   - Remove samples where the gene of interest has zero expression.
   - Print the percentage of samples with non-zero expression for the gene of interest.

5. **Calculate Correlations:**
   - Compute Pearson correlation coefficients for the gene of interest.
   - Return and print the top 500 most positively and negatively correlated genes.

#### Usage Instructions:

- Modify the `cell_type`, `tissue_type`, and `gene_of_interest` variables as needed.
- Use `get_coexpression_matrix(gene, tissue, cell_type, k=500)` to obtain top correlated genes.

#### Notes:

- Ensure access to the appropriate AnnData formatted dataset.
- Experiment with different parameters to explore gene expression correlations.


### Downloading Dependencies

To run this notebook, ensure you have the necessary libraries installed:

- `cellxgene_census` for accessing gene expression data.
- `pandas` for data manipulation and analysis.
- `numpy` for numerical operations.
- `scipy.stats` for statistical calculations, including Pearson correlation (`pearsonr`).


In [1]:
# %pip install cellxgene-census pandas numpy scipy
%pip install cellxgene-census

  Using cached pandas-2.2.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (19 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 5.8 MB/s eta 0:00:00a 0:00:01
Using cached pandas-2.2.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.0 MB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.3.1
    Uninstalling fsspec-2024.3.1:
      Successfully uninstalled fsspec-2024.3.1
  Attempting uninstall: pandas
    Found existing installation: pandas 1.4.4
    Uninstalling pandas-1.4.4:
      Successfully uninstalled pandas-1.4.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 2.19.2 requires fsspec[http]<=2024.3.1,>=2023.1.0, but you have fsspec 2024.5.0 which is incompatible.
gwaslab 3.4.24 requires pandas<1.5,>=1.3, but you have pandas 2.2.2 which is incompatible.
torchvision 0.18.1+cu

### Importing Required Libraries

This code cell imports necessary libraries for data analysis:

- `cellxgene_census`: Imports functionality for working with cellxgene data.
- `pandas` (`pd`): Imports the Pandas library for data manipulation and analysis.
- `numpy` (`np`): Imports NumPy for numerical computing operations.
- `pearsonr` from `scipy.stats`: Imports the `pearsonr` function specifically for computing Pearson correlation coefficients.


In [2]:
import cellxgene_census
import pandas as pd
import numpy as np
from scipy.stats import pearsonr

/home/abdu/.local/lib/python3.10/site-packages/cupy/_environment.py:445: UserWarning: 
--------------------------------------------------------------------------------

  CuPy may not function correctly because multiple CuPy packages are installed
  in your environment:

    cupy, cupy-cuda11x

  Follow these steps to resolve this issue:

    1. For all packages listed above, run the following command to remove all
       existing CuPy installations:

         $ pip uninstall <package_name>

      If you previously installed CuPy via conda, also run the following:

         $ conda uninstall cupy

    2. Install the appropriate CuPy package.
       Refer to the Installation Guide for detailed instructions.

         https://docs.cupy.dev/en/stable/install.html

--------------------------------------------------------------------------------

  warnings.warn(f'''


### Function to Get Co-Expression Matrix

The following code defines a function `get_coexpression_matrix` that performs the following steps:

1. **Open and Query Data:**
   - Uses `cellxgene_census.open_soma()` to access the gene expression data.
   - Queries data for the organism 'Homo sapiens' and filters by a specific cell type (`cell_type`) using `obs_value_filter=f"cell_type == '{cell_type}'"`.

2. **Ensure Correct Gene Names and Filter Genes:**
   - Checks and sets gene names (`adata.var['feature_id']`).
   - Filters out genes with zero expression values (`adata.X > 0`).

3. **Convert Data to DataFrame:**
   - Converts the filtered expression data into a Pandas DataFrame (`df_expression`).

4. **Filter Samples Based on Gene of Interest Expression:**
   - Filters out samples where the gene of interest has zero expression.
   - Calculates and prints the percentage of samples with non-zero expression for the gene of interest.

5. **Calculate Pearson Correlations:**
   - Computes Pearson correlation coefficients between the gene of interest and other genes in `df_expression`.
   - Sorts and returns the top 500 most positively and negatively correlated genes.

The function is then run with an example gene (`ENSG00000140718`) and the specified tissue and cell type (`Adipose` and `adipocyte`). The top 10 most positively and negatively correlated genes are printed.

#### Usage Instructions:

- Call `get_coexpression_matrix(gene, tissue, cell_type, k=500)` with your desired gene, tissue type, and cell type.
- The function will return two lists: the top 500 most positively and negatively correlated genes.
- Modify `k` to adjust the number of top correlations returned.

Example:
```python
top_positive, top_negative = get_coexpression_matrix('ENSG00000140718', 'Adipose', 'adipocyte', k=500)


In [35]:
def get_coexpression_matrix(gene, tissue, cell_type, k=500):
    with cellxgene_census.open_soma() as census:
        # Query the data for a specific organism and cell types
        adata = cellxgene_census.get_anndata(
            census=census,
            organism="Homo sapiens",
            obs_value_filter=f"cell_type == '{cell_type}'", # use obs_value_filter=f"tissue_general == '{tissue}'" if you want to filter with tissue type
            # obs_value_filter=f"tissue_general == '{tissue}'",
            column_names={"obs": ["assay", "cell_type", "tissue", "tissue_general", "suspension_type", "disease"]},
        )

        # Ensure the gene names are set correctly
        if 'feature_id' in adata.var.columns:  # Adjust column name as needed
            adata.var_names = adata.var['feature_id']
        else:
            print("Gene names column 'feature_id' not found in var DataFrame")

        # Filter out genes with zero expression values
        gene_expression_sum = np.array((adata.X > 0).sum(axis=0)).flatten()
        adata_filtered = adata[:, gene_expression_sum > 0]
        genes = adata_filtered.var['feature_id']
        # Convert the filtered expression data to a DataFrame
        df_expression = pd.DataFrame(adata_filtered.X.toarray(), columns=genes)

        # Check if the gene of interest is in the dataset
        if gene in df_expression.columns:
            # Filter out samples where the gene of interest is not expressed (expression value = 0)
            non_zero_samples = df_expression[df_expression[gene] > 0]
            
            # Calculate percentage of samples with non-zero expression for the gene of interest
            total_samples = df_expression.shape[0]
            non_zero_sample_count = non_zero_samples.shape[0]
            non_zero_percentage = (non_zero_sample_count / total_samples) * 100

            print(f"Total samples: {total_samples}")
            print(f"Samples with non-zero expression for '{gene}': {non_zero_sample_count} ({non_zero_percentage:.2f}%)")

            # Calculate Pearson correlation coefficients and p-values
            correlations = {}
            for g in non_zero_samples.columns:
                if g != gene:
                    corr, p_value = pearsonr(non_zero_samples[gene], non_zero_samples[g])
                    if p_value < 0.05:
                        correlations[g] = corr
                        
            # Sort correlations
            sorted_correlations = sorted(correlations.items(), key=lambda x: x[1], reverse=True)
            top_positive = sorted_correlations[:k]
            top_negative = sorted_correlations[-k:]

            return top_positive, top_negative, genes
        else:
            print(f"Gene of interest '{gene}' not found in the dataset.")
            return [], []

### Selecting Tissue or Cell Type and Gene of Interest

The below code cell sets variables to specify the tissue or cell type and a specific gene for analysis:


In [34]:
# gene_of_interest = 'ENSG00000140718' #FTO
gene_of_interest = 'ENSG00000177508' #IRX3
tissue_type = 'Adipose'
cell_type = 'preadipocyte'

### Running the Co-Expression Matrix Function

The following code calls the `get_coexpression_matrix` function to obtain the top 500 most positively and negatively correlated genes for a specified gene of interest, tissue type, and cell type.


In [36]:
top_positive, top_negative, all_genes = get_coexpression_matrix(gene_of_interest, tissue_type, cell_type, k=500)
len(all_genes)

The "stable" release is currently 2024-07-01. Specify 'census_version="2024-07-01"' in future calls to open_soma() to ensure data consistency.


Total samples: 104964
Samples with non-zero expression for 'ENSG00000177508': 3310 (3.15%)


/tmp/ipykernel_2941079/2144300635.py:42: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  corr, p_value = pearsonr(non_zero_samples[gene], non_zero_samples[g])


28599

### Printing Top Correlated Genes

The following code prints the top 10 most positively correlated genes with the specified gene of interest. Each gene is listed along with its Pearson correlation coefficient.

In [37]:
print("Top 10 most positively correlated genes:")
for gene, corr in top_positive[:10]:
    print(f"{gene}: correlation = {corr:.3f}")


Top 10 most positively correlated genes:
ENSG00000184731: correlation = 0.235
ENSG00000277159: correlation = 0.194
ENSG00000275265: correlation = 0.186
ENSG00000251194: correlation = 0.161
ENSG00000132297: correlation = 0.160
ENSG00000171903: correlation = 0.160
ENSG00000231764: correlation = 0.160
ENSG00000256812: correlation = 0.160
ENSG00000260271: correlation = 0.160
ENSG00000236485: correlation = 0.154


The following code prints the top 10 most negatively correlated genes with the specified gene of interest. Each gene is listed along with its Pearson correlation coefficient.

In [38]:
print("\nTop 10 most negatively correlated genes:")
for gene, corr in top_negative[:10]:
    print(f"{gene}: correlation = {corr:.3f}")


Top 10 most negatively correlated genes:
ENSG00000272870: correlation = 0.034
ENSG00000186432: correlation = 0.034
ENSG00000162923: correlation = 0.034
ENSG00000181038: correlation = 0.034
ENSG00000116984: correlation = 0.034
ENSG00000165914: correlation = 0.034
ENSG00000150637: correlation = 0.034
ENSG00000155090: correlation = 0.034
ENSG00000160285: correlation = 0.034
ENSG00000134243: correlation = 0.034


In [39]:
import pickle
import os

ensembl_to_hgnc_map = pickle.load(open("./data/ensembl_to_hgnc.pkl", "rb"))

In [40]:
top_positive_hgnc = [(ensembl_to_hgnc_map.get(gene, gene), corr) for gene, corr in top_positive]
top_negative_hgnc = [(ensembl_to_hgnc_map.get(gene, gene), corr) for gene, corr in top_negative]
all_genes_hgnc = [ensembl_to_hgnc_map.get(gene, gene) for gene in all_genes]

In [41]:
all_genes_hgnc[:10]

['TSPAN6',
 'TNMD',
 'DPM1',
 'SCYL3',
 'FIRRM',
 'FGR',
 'CFH',
 'FUCA2',
 'GCLC',
 'NFYA']

In [42]:
import gseapy as gp 

library = "GO_Biological_Process_2023"
organism = "Human"

res = gp.enrichr(gene_list=[gene[0] for gene in top_positive_hgnc],
                                gene_sets=library,
                                background=all_genes_hgnc,
                                organism=organism,
                                outdir=None).results
res.drop("Gene_set", axis=1, inplace=True)
res.insert(1, "ID", res["Term"].apply(
    lambda x: x.split("(")[1].split(")")[0]))
res["Term"] = res["Term"].apply(lambda x: x.split("(")[0])
# res = res[res["Adjusted P-value"] < 0.05]

In [43]:
res.head()

,Term,ID,P-value,Adjusted P-value,Old P-value,Old adjusted P-value,Odds Ratio,Combined Score,Genes
0,Cytoplasmic Translation,GO:0002181,1.116099e-37,2.037996e-34,0,0,37.029617,3150.694935,RPL5;RPLP1;RPL12;RPLP0;RPL11;RPS14;RPS16;RPS15...
1,Peptide Biosynthetic Process,GO:0043043,2.814802e-29,1.948846e-26,0,0,18.315849,1204.085271,RPL5;RPLP1;RPL12;RPLP0;RPL11;RPS14;RPS16;RPS15...
2,Translation,GO:0006412,3.201828e-29,1.948846e-26,0,0,13.261380,870.095659,RPL5;RPLP1;RPL12;RPLP0;RPL11;MRPL58;RPS14;RPS1...
3,Macromolecule Biosynthetic Process,GO:0009059,7.738994e-29,3.532851e-26,0,0,15.733413,1018.403336,RPL5;ALAS1;RPLP1;RPL12;RPLP0;RPL11;RPS14;RPS16...
4,Gene Expression,GO:0010467,5.626074e-23,2.054642e-20,0,0,9.311207,477.032177,RPL5;RPLP1;RPL12;RPLP0;RPL11;HNRNPU;HSPD1;RBM3...


In [45]:
# case insensitive search
res[res["Term"].str.contains("adipose", case=False)]

,Term,ID,P-value,Adjusted P-value,Old P-value,Old adjusted P-value,Odds Ratio,Combined Score,Genes
789,Adipose Tissue Development,GO:0060612,0.190788,0.423737,0,0,5.117143,8.477034,XBP1


In [14]:

# Negative correlation
res_neg = gp.enrichr(gene_list=[gene[0] for gene in top_negative_hgnc],
                                gene_sets=library,
                                background=all_genes_hgnc,
                                organism=organism,
                                outdir=None).results
# res_neg.drop("Gene_set", axis=1, inplace=True)
# res_neg.insert(1, "ID", res_neg["Term"].apply(
#     lambda x: x.split("(")[1].split(")")[0]))
# res_neg["Term"] = res_neg["Term"].apply(lambda x: x.split("(")[0])
# res_neg = res_neg[res_neg["Adjusted P-value"] < 0.05]

In [17]:
res_neg[res_neg["Term"].str.contains("fat", case=False)]

,Gene_set,Term,P-value,Adjusted P-value,Old P-value,Old adjusted P-value,Odds Ratio,Combined Score,Genes
82,GO_Biological_Process_2023,Negative Regulation Of Fat Cell Differentiatio...,0.009124,0.188119,0,0,7.582738,35.615038,FOXO1;FERMT2;RUNX1T1
123,GO_Biological_Process_2023,Regulation Of Fat Cell Differentiation (GO:004...,0.018925,0.255755,0,0,4.133121,16.397186,ADIPOQ;FOXO1;FERMT2;RUNX1T1
194,GO_Biological_Process_2023,Long-Chain Fatty Acid Transport (GO:0015909),0.038317,0.339600,0,0,6.968445,22.730127,FABP4;CD36
213,GO_Biological_Process_2023,Regulation Of Fatty Acid Biosynthetic Process ...,0.044642,0.353666,0,0,6.362144,19.780399,MLXIPL;CEACAM1
231,GO_Biological_Process_2023,Fat Cell Differentiation (GO:0045444),0.051802,0.353666,0,0,3.724039,11.024382,ADIPOQ;ZFPM2;FOXO1
354,GO_Biological_Process_2023,Long-Chain Fatty Acid Catabolic Process (GO:00...,0.078480,0.353666,0,0,14.610822,37.183325,ACADL
388,GO_Biological_Process_2023,Fatty Acid Metabolic Process (GO:0006631),0.084274,0.353666,0,0,2.483666,6.143813,GPAM;ADIPOQ;CD36;ACACB
399,GO_Biological_Process_2023,Fatty Acid Transport (GO:0015908),0.089250,0.353666,0,0,4.179461,10.098903,FABP4;CD36
479,GO_Biological_Process_2023,Brown Fat Cell Differentiation (GO:0050873),0.103249,0.357831,0,0,10.435729,23.695508,ADIPOQ
484,GO_Biological_Process_2023,Unsaturated Fatty Acid Biosynthetic Process (G...,0.103249,0.357831,0,0,10.435729,23.695508,FADS3
